In [1]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
from peft import LoraConfig, get_peft_model

/mnt/45b9faff-45f3-43f2-903f-9b92a9a6338c/major-project/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("your_nepali_english_dataset")

train_dataset = dataset['train']
val_dataset = dataset['validation']


In [8]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50", src_lang="ne_NP", tgt_lang="en_XX")

In [9]:
with open('mbart_architecture.txt','w') as file:
    file.writelines(str(model))

In [10]:
lora_config = LoraConfig(
    r=8,  # Low-rank matrix dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to specific layers in the attention mechanism
    bias="none",
)

model = get_peft_model(model, lora_config)


In [ ]:
def preprocess_function(examples):
    inputs = [ex["nepali_text"] for ex in examples]
    targets = [ex["english_text"] for ex in examples]
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, return_tensors="pt", padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, return_tensors="pt", padding="max_length").input_ids
    
    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart-nepali-english-lora",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  
    logging_dir='./logs',
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
model.push_to_hub("Strange18/mbart-nepali-english-lora")
tokenizer.push_to_hub("Strange18/mbart-nepali-english-lora")
